# 8장. 수치형

## 8.1 수치형 데이터로 변환


#### SQL의 CAST함수

자료형을 변환하는 함수
> * SELECT CAST(29.8 AS INT)  
        * 결과 : 29 
* SELECT CAST(3 AS VARCHAR) + 'age' 
         * 결과 : 3age 
         * 위와 같이 숫자와 문자를 붙일 때도 CAST함수 사용

#### 40000/3 을 다양한 수치 데이터로 변환해보자.

In [8]:
import pandas as pd
import numpy as np

In [1]:
# 자료형을 확인
type(40000/3)

float

In [2]:
# 정수형으로 변환
int(40000/3)

13333

In [3]:
# 부동 소수점형으로 변환
float(40000/3)

13333.333333333334

In [5]:
df = pd.DataFrame({'value':[40000/3]})
df

,value
0,13333.333333


In [6]:
# 자료형 확인
df.dtypes

value    float64
dtype: object

In [10]:
# 정수형으로 변환
df['value'].astype('int8')
df['value'].astype('int16')
df['value'].astype('int32')
df['value'].astype('int64')

0    13333
Name: value, dtype: int64

In [12]:
# 부동 소수점형으로 변환
df['value'].astype('float16')
df['value'].astype('float32')
df['value'].astype('float64')
df['value'].astype('float128')

0    13333.333333
Name: value, dtype: float128

In [13]:
# 다음과 같이 파이썬의 자료형을 지정할 수 있다.
df['value'].astype(int)

0    13333
Name: value, dtype: int64

In [14]:
df['value'].astype(float)

0    13333.333333
Name: value, dtype: float64

----------------
## 8.2 대수화를 이용한 비선형 변화
머신러닝 모델을 전처리 할 때 앞으로 적용할 머신러닝 모델이 선형 모델 , 비선형 모델 중 어느쪽을 가정하는지 확인해야 한다.

* 선형모델
    * 간단하고 계산이 빠르며 예측 계산의 근거를 알기 쉽지만 표현력이 떨어지는 단점. (y=ax+b) 형태

* 대수화
    * 입력값을 대수로 변환하는 처리. (ex. 로그변환)
    * 대수화는 값이 커질수록 변화량을 작게 할 때 유용하다.
        * ex) 나이로 키를 예측하는 모델에서, 같은 한 살 차이라도 10세와 11세의 차이는 크고 50세와 51세의 차이는 작다.
        

In [27]:
reserve_tb=pd.read_csv('./data/reserve.csv',encoding='UTF-8')

In [28]:
reserve_tb['total_price'][:10]

0     97200
1     20600
2     33600
3    194400
4     68100
5     36000
6    103500
7      6000
8     68400
9    320400
Name: total_price, dtype: int64

* lambda함수를 이용해서 log 변환

In [29]:
reserve_tb['total_price_log']=reserve_tb['total_price'].apply(lambda x: np.log10(x/1000 +1))

In [30]:
reserve_tb['total_price_log'][:10]

0    1.992111
1    1.334454
2    1.539076
3    2.290925
4    1.839478
5    1.568202
6    2.019116
7    0.845098
8    1.841359
9    2.507046
Name: total_price_log, dtype: float64

------------
## 8.3 범주화를 이용한 비선형 변화
범주화란 수치를 다수의 플래그 값 (TRUE 또는 FALSE값을 가진 값)으로 변환하는 것을 말한다.
* ex) 연령을 0~9일때, 10~19일때, 20~59일때, 60세 이상일 때의 플래그로 변환.

In [45]:
customer_tb=pd.read_csv('./data/customer.csv',encoding='UTF-8')

#### 고객 테이블 나이를 10단위의 번주형으로 추가해보자.
floor함수는 매개변수의 소수점 이하를 버린다. 
* astype() 으로 형 변환

In [46]:
customer_tb['age_rank']=(np.floor(customer_tb['age']/10)*10).astype('category')

In [47]:
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,age_rank
0,c_1,41,man,35.092193,136.512347,40.0
1,c_2,38,man,35.325076,139.410551,30.0
2,c_3,49,woman,35.120543,136.511179,40.0
3,c_4,43,man,43.034868,141.240314,40.0
4,c_5,31,man,35.102661,136.523797,30.0


----------
## 8.4 정규화
중회귀분석이나 클러스터링과 같은 머신러닝은 일반적으로 여러 열의 값을 이용한다. <br>
이러한 머신러닝을 이용할 때 열마다 가질 수 있는 값의 범위가 크게 차이가 나면 문제가 생긴다. <br>
예를들면 중회귀 분석 모델에 정형화(regularization)을 적용할 경우. <br>
* 정형화란 머신러닝 모델을 학습할 때 과학습이 발생하는 것을 방지하기 위한 장치.

#### 정규화 (normalization)
값이 가질 수 있는 범위를 통일하는 변환 처리
> 두 가지 정규화 방법 <br>
1) 평균 0, 분산 1로 변환하는 정규화
    * (입력값 - 입력값의 평균값)/(입력값의 표준편차)
2) 최솟값0, 최댓값1로 변환하는 정규화
    * (입력값 - 입력값의 최솟값) / (입력값의 최댓값 - 입력값의 최솟값)

정규화 방식은 우선 튀는 값이 존재하면 평균 0, 분산 1로 변환하는 정규화를 사용하고, <br>
튀는 값이 있지 않으면 최소 0, 최대 1로 변환하는 정규화를 사용한다.
    

#### people_num과 total_price를 평균 0, 분산1의 분포로 정규화해보자.

In [49]:
from sklearn.preprocessing import StandardScaler

In [52]:
reserve_tb['people_num'].dtype

dtype('int64')

In [53]:
# 소수점 이하를 다룰 수 있도록 float형으로 변환
reserve_tb['people_num'] = reserve_tb['people_num'].astype(float)

In [54]:
reserve_tb['people_num'].dtype

dtype('float64')

In [55]:
# 정규화를 실행할 오브젝트를 생성한다.
ss = StandardScaler()

In [57]:
# fit_transform함수는 fit함수(정규화하기 위한 준비 계산)와 
# transform함수 (준비된 정보에서 정규화 변환 처리를 실행) 를 모두 실행한다.
result = ss.fit_transform(reserve_tb[['people_num','total_price']])

/Users/suhyun/anaconda3/envs/suhyun/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/suhyun/anaconda3/envs/suhyun/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [71]:
reserve_tb[['people_num','total_price']].head()

,people_num,total_price
0,4.0,97200
1,2.0,20600
2,2.0,33600
3,4.0,194400
4,3.0,68100


In [64]:
result

array([[ 1.30070909, -0.05319398],
       [-0.48375308, -0.74782231],
       [-0.48375308, -0.62993499],
       ...,
       [-0.48375308, -0.25632289],
       [ 1.30070909,  3.96222931],
       [-1.37598416, -0.53471832]])

* 두개의 리스트로 묶여있을 때, for문으로 각 리스트별 값에 연산을 적용할 수 있음

In [80]:
result[0][0]

1.3007090857379016

In [81]:
reserve_tb['people_num_normalized']=[x[0] for x in result]
reserve_tb['total_price_normalized']=[x[1] for x in result]

In [97]:
[x[0] for x in result][:10]

[1.3007090857379016,
 -0.48375308124890476,
 -0.48375308124890476,
 1.3007090857379016,
 0.40847800224449843,
 0.40847800224449843,
 -1.375984164742308,
 -1.375984164742308,
 0.40847800224449843,
 1.3007090857379016]

----------
## 8.5 예욋값 제거
여기서는 정규분포를 전제로 한 예욋값 검출 방법과 <br>
계산식에 의존하지 않고 데이터를 가시화하여 예욋값을 직접 제거하는 방법을 설명.

> 가장 간단하면서 많이 사용되는 방법
* 정규분포를 전제로 한 예욋값 검출 방법은 표준편차의 일정 배수 이상 떨어진 값을 평균값에서 제거하는 방법
    * 정규분포에 따른 값은 평균값에서 표준편차의 세 배 이내 범위에 99.73%의 값이 있으므로, 발생할 확률이 0.27% 이하인 값을 예욋값으로 보는 것이 좋다.

#### total_price의 평균값에서 표준편차의 세 배 이내 값만 있도록 수정
|(값) - (값의 평균)| / (값의 표준편차) <= 3

In [86]:
reserve_tb=reserve_tb[(abs(reserve_tb['total_price']-np.mean(reserve_tb['total_price'])) /
           np.std(reserve_tb['total_price']) <=3)].reset_index()

-----------
## 8.6 주성분 분석을 이용한 차원 압축
주성분 분석(principal component analysis)은 요소 사이의 상관관계를 배제하여 되도록 정보의 손실 없이 새로운 요소(축)를 정의하는 분석 방법.

> 기여율
* 주성분 분석의 평가 지표
    * 기여율은 지정한 주성분 축 하나가 데이터 변동의 몇 할을 설명할 수 있는지 나타낸 값
        * 이 값이 높을수록 차원을 압축했을 때의 정보 손실이 적음
        
* 주성분 분석을 이용한 차원 압축은 지정한 차원수로 압축할 수 있다.
    * 압축 후의 차원 수는 기여율의 합곗값(누적 기여율) 을 기준으로 정한다. (일반적으로 누적 기여율이 90% 이상인 차원수를 많이 선택함.)

> PCA의 효과
* 선형 모델로 다중공선성이라는 변수 사이에 강한 상관관계가 있어 과학습이 생기는 것을 방지할 수 있음 .

In [3]:
production_tb=pd.read_csv('./data/production.csv',encoding='UTF-8')
production_tb.head()

,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False


In [90]:
from sklearn.decomposition import PCA

In [91]:
# n_components에 주성분 분석으로 변환 후의 차원수를 지정한다.
pca = PCA(n_components=2)

In [92]:
# pca에 주성분 분석 변환 매개변수가 저장되어 반환값에 주성분 분석 후의 값이 반환된다.
pca_values=pca.fit_transform(production_tb[['length','thickness']])

In [93]:
pca_values

array([[  76.96838157,  -13.38906936],
       [-112.11469337,   -8.24884796],
       [ -76.1994339 ,   11.19027127],
       ...,
       [  31.12100559,   15.48152593],
       [-117.87675543,    2.4361334 ],
       [   4.80243541,  -15.32174872]])

In [94]:
# 누적 기여율과 기여율을 확인한다.
print('누적 기여율:{0}'.format(sum(pca.explained_variance_ratio_)))
print('각 차원의 기여율:{0}'.format(pca.explained_variance_ratio_))

누적 기여율:1.0
각 차원의 기여율:[0.97897794 0.02102206]


In [95]:
# predict 함수를 이용하여 동일한 차원 압축 처리를 실행.
pca_newvalues=pca.transform(production_tb[['length','thickness']])

In [96]:
pca_newvalues

array([[  76.96838157,  -13.38906936],
       [-112.11469337,   -8.24884796],
       [ -76.1994339 ,   11.19027127],
       ...,
       [  31.12100559,   15.48152593],
       [-117.87675543,    2.4361334 ],
       [   4.80243541,  -15.32174872]])

-----
## 8.7 수치의 보완

#### 결손의 종류
* MCAR(Missing Completely At Random): 우연히 발생한 완전 무작위 결손.
* MAR(Missing At Random) : 결손된 항목 데이터와 관계없이 다른 항목 데이터에 의존한 결손.
* MNAR(Missing Not At Random ): 결손된 항목 데이터에 의존한 결손. 

#### 데이터 보완 방법
1. 정수로 보완한다.
    * 임의의 값을 결손된 값의 보완값으로 이용하는 방법. 
2. 집곗값으로 보완한다.
    * 결손되지 않은 값의 평균값, 중앙값, 최솟값, 최댓값 등을 계산하여 데이터가 없는 값의 보완값으로 이용하는 방법.
3. 결손되지 않은 데이터를 이용한 예측값으로 보완한다.
    * 결손이 발생하지 않은 열의 값과 일부 결손이 발생한 열의 값의 관계로 결손된 값을 예측하여 보완하는 방법.
4. 시간 관계를 이용해 보완한다.
    * 결손이 발생한 데이터의 앞 뒤 데이터로 결손값을 예측하여 보완하는 방법.
5. 다중대입법으로 보완한다.
    * 보완한 데이터셋을 여러 개 만들어 각 데이터셋 해석을 하는 것. 이렇게 생성된 여러 결과를 통합하여 바이어스가 적은 결과를 얻는다.
 6. 최대 가능도로 보완한다.
     * 잠재 변수를 도입하여 EM 알고리즘을 사용해 (보완한 결과 데이터가 다변량 정규분포를 따르는) 가능도를 최대화하여 결손값을 보완한다.
     
=> 일반적으로 MCAR, MAR은 다중대입법이나 최대가능도를 사용한다.

### Q1. 결손 레코드 제거
파이썬 dropna 함수는 nan을 결손값으로 인식하지만, None은 결손값으로 인식하지 않는다.

In [14]:
production_miss_num=pd.read_csv('./data/production_missing_num.csv',encoding='UTF-8')

In [15]:
production_miss_num[24:28]

,type,length,thickness,fault_flg
24,B,202.549369,None,False
25,D,223.806149,19.65076599420279,False
26,B,263.844324,34.66425112689404,False
27,B,169.690602,None,False


In [37]:
# replace 함수로 None을 nan으로 변환한다.
# (None을 지정할 때는 문자열로 지정해야 한다.)
production_miss_num.replace('None',np.nan, inplace=True)

In [25]:
production_miss_num[24:28]

,type,length,thickness,fault_flg
24,B,202.549369,NaN,False
25,D,223.806149,19.65076599420279,False
26,B,263.844324,34.66425112689404,False
27,B,169.690602,NaN,False


In [26]:
# dropna 함수로 thickness에 nan이 포함된 레코드를 제거한다.
production_miss_num.dropna(subset=['thickness'],inplace=True)

* subset을 지정하지않으면 모든 열에 있는 nan값을 제거하게 되므로 주의!

In [27]:
production_miss_num[24:28]

,type,length,thickness,fault_flg
25,D,223.806149,19.65076599420279,False
26,B,263.844324,34.66425112689404,False
28,A,131.849213,4.721600725740606,True
29,E,128.989765,16.52324107337661,False


### Q2. 정수 보완
결손된 thickness값을 1로 보완한다.
* SQL에서 결손값 보완은 COALESCE() 함수를 사용

In [31]:
production_miss_num.replace('None', np.nan, inplace=True)

In [32]:
production_miss_num['thickness'].fillna(1, inplace=True)

In [34]:
production_miss_num[24:28]

,type,length,thickness,fault_flg
24,B,202.549369,1,False
25,D,223.806149,19.65076599420279,False
26,B,263.844324,34.66425112689404,False
27,B,169.690602,1,False


### Q3. 평균값 보완
결손된 thickness의 값을 결손되지 않은 thickness의 평균값으로 보완한다.

In [39]:
# thickness를 수치형으로 변환한다 (None이 섞여 있어 수치형이 아닌 상태이다.)
production_miss_num['thickness']=production_missing_num['thickness'].astype('float64')

In [40]:
production_miss_num['thickness'].dtype

dtype('float64')

In [42]:
thickness_mean=production_miss_num['thickness'].mean()

In [43]:
production_miss_num[24:28]

,type,length,thickness,fault_flg
24,B,202.549369,NaN,False
25,D,223.806149,19.650766,False
26,B,263.844324,34.664251,False
27,B,169.690602,NaN,False


In [44]:
production_miss_num['thickness'].fillna(thickness_mean, inplace=True)

In [45]:
production_miss_num[24:28]

,type,length,thickness,fault_flg
24,B,202.549369,19.470386,False
25,D,223.806149,19.650766,False
26,B,263.844324,34.664251,False
27,B,169.690602,19.470386,False


### Q4. PMM(predictive mean matching)을 이용한 다중대입
PMM에 대한 간략한 설명
1. 결손 데이터를 제외한 데이터에서 결손 데이터를 예측하는 회귀 모델을 구성한다.
2. 구성된 회귀 모델의 계수와 오차 분산의 분포를 계산한다.
3. 계수와 오차 분산의 분포에서 새로운 계수와 오차 분산의 값을 생성한다.
4. 3.에서 생성한 계수와 오차 분산값에 다른 회귀 모델에서 예측값을 계산한다.
5. 결손되지 않은 관측 데이터 중 예측값에 가장 가까운 데이터를 보완값으로 선택한다.
6. 데이터를 보완하여 새롭게 구성한 회귀 모델의 계수와 오차 분산의 분포를 계산하고 3.으로 돌아간다.

In [ ]:
pip install fancyimpute

    100% |████████████████████████████████| 747kB 49kB/s ta 0:00:01
    100% |████████████████████████████████| 7.1MB 235kB/s ta 0:00:01
    100% |████████████████████████████████| 378kB 194kB/s ta 0:00:01
    100% |████████████████████████████████| 120.8MB 170kB/s ta 0:00:011  12% |███▉                            | 14.5MB 183kB/s eta 0:09:41    46% |███████████████                 | 56.7MB 155kB/s eta 0:06:54    64% |████████████████████▋           | 77.6MB 31kB/s eta 0:22:34    72% |███████████████████████         | 87.0MB 47kB/s eta 0:11:53    73% |███████████████████████▋        | 89.1MB 48kB/s eta 0:11:02    82% |██████████████████████████▍     | 99.8MB 301kB/s eta 0:01:10    83% |██████████████████████████▉     | 101.4MB 352kB/s eta 0:00:55
    100% |████████████████████████████████| 133kB 433kB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 575kB/s ta 0:00:01
    100% |████████████████████████████████| 1.6MB 746kB/s ta 0:00:01    88% |████████████████████████████

    100% |████████████████████████████████| 829kB 880kB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 869kB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 921kB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 2.0MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 1.0MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 1.3MB/s ta 0:00:01
  Stored in directory: /Users/suhyun/Library/Caches/pip/wheels/0e/65/31/fff6a8fa9d1df4c6204f5a9059340347d2085b971b67d3f0a0
  Stored in directory: /Users/suhyun/Library/Caches/pip/wheels/a3/92/31/c3f8864714e9938396c3a68d8c542531f7e2d7862bb750b2e3
  Stored in directory: /Users/suhyun/Library/Caches/pip/wheels/4b/7d/90/39ff7dca0e5c06740afc993ab4209a1719fc1d616daf7af040
  Stored in directory: /Users/suhyun/Library/Caches/pip/wheels/68/3f/24/e9c75d426f600634cdac68321184ba06fdc4ab2d189b5c4541
  Stored in directory: /Users/suhyun/Library/Caches/pip/wheels/96/20/ac/9f1d164f7d81787cd6f4401b1

In [10]:
from fancyimpute import MICE

ImportError: cannot import name 'MICE' from 'fancyimpute' (/Users/suhyun/anaconda3/envs/suhyun/lib/python3.7/site-packages/fancyimpute/__init__.py)

In [16]:
production_miss_num.replace('None',np.nan, inplace=True)

In [17]:
# mice함수를 이용하기 위해 자료형을 변환한다. 
production_miss_num['thickness']=production_miss_num['thickness'].astype('float64')
production_miss_num['type']=production_miss_num['type'].astype('category')
production_miss_num['fault_flg']=production_miss_num['fault_flg'].astype('category')

In [21]:
# 더미변수화
production_dummy_flg=pd.get_dummies(production_miss_num[['type','fault_flg']], drop_first=True)

In [23]:
production_dummy_flg.head()

,type_B,type_C,type_D,type_E,fault_flg_True
0,0,0,0,1,0
1,0,0,1,0,0
2,0,0,0,1,0
3,1,0,0,0,0
4,1,0,0,0,0


In [25]:
pd.concat([production_miss_num[['length','thickness']], production_dummy_flg],axis=1).head()

,length,thickness,type_B,type_C,type_D,type_E,fault_flg_True
0,274.027383,40.241131,0,0,0,1,0
1,86.319269,16.906715,0,0,1,0,0
2,123.940388,1.018462,0,0,0,1,0
3,175.554886,16.414924,1,0,0,0,0
4,244.934740,29.061081,1,0,0,0,0
